In [ ]:
import copy

import prody
from rdkit import Chem
from rdkit.Chem import Draw

from rgroup import Rmol, build_molecules, fix_receptor, optimise_in_receptor, RGroups, sort_conformers

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('5NEV.pdb1')
rec = sys.select('not (nucleic or hetatm) and not water')
mol = sys.select('resname 72L')

prody.writePDB('rec.pdb', rec)
# prody.writePDB('lig.pdb', mol)

In [ ]:
!wget 'https://models.rcsb.org/v1/5nev/ligand?auth_seq_id=301&label_asym_id=E&encoding=sdf&filename=5nev_E_72L.sdf' -O lig.sdf
!obabel lig.sdf -O ligh.sdf -h -p 7
# ^manually modify the files if need be

In [ ]:
# load the ligand
# rdkit doesn't do a good job with protonation, 
#init_mol = Chem.MolFromPDBFile('lig.pdb', removeHs=False)
#init_mol = Chem.AddHs(init_mol, addCoords=True)

init_mol = Chem.SDMolSupplier('ligh.sdf', removeHs=False)[0]

# get the R group representation of the rdkit Mol
rinit_mol = Rmol(init_mol)

In [ ]:
# select the atom to be replaced with the R group
rinit_mol.rep2D(idx=True, size=(500, 500))

In [ ]:
# look up r groups and select
RGroups

In [ ]:
# build a list of molecules
rmols = build_molecules(rinit_mol, [40], RGroups)

In [ ]:
rmols[0].rep3D()

In [ ]:
Draw.MolsToGridImage([mol.rep2D(rdkit_mol=True) for mol in rmols], subImgSize=(400, 400))

In [ ]:
# take the first molecule
rmol = rmols[0]
rmol.rep2D()

In [ ]:
rmol.toxicity()

In [ ]:
rmol.rep3D()

In [ ]:
rmol.generate_conformers(num_conf=20, 
                         minimum_conf_rms=0.5, 
                         flexible=[39, 18, 40, 16, 17, 19, 5, 6, 36, 2, 1, 33, 3, 4, 35, 34, 15, 38])

In [ ]:
rmol.rep3D(template=True)

In [ ]:
rmol.GetNumConformers()

In [ ]:
# fix the receptor file 
fix_receptor("rec.pdb", "rec_h.pdb")
# load back into prody
rec_fix = prody.parsePDB("rec_h.pdb")

In [ ]:
# filter by the protein clashes
rmol.rep3D(prody=rec_fix)

In [ ]:
rmol.removeConfsClashingWithProdyProt(rec_fix)

In [ ]:
rmol.rep3D(prody=rec_fix)

In [ ]:
opt_mol, energies = optimise_in_receptor(
     ligand=rmol, 
     receptor_file="rec_h.pdb", 
     ligand_force_field="openff", 
     use_ani=False,
     sigma_scale_factor=0.8,
     relative_permittivity=4
 )

In [ ]:
type(opt_mol)

opt_mol.rep3D()

In [ ]:
opt_mol.to_file("optimised_corrds.pdb")

In [ ]:
print(energies)

In [ ]:
final_mol, final_energies = sort_conformers(ligand=opt_mol, energies=energies, energy_range=5)

In [ ]:
final_mol.to_file("best_conformers.pdb")

In [ ]:
print(final_energies)